In [ ]:
!pip install -q transformers

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
senti_map = {"POSITIVE" : 1, "NEGATIVE" : -1}

def sent_score(chunk):
  senti = sentiment_pipeline(chunk)[0]
  return senti_map[senti['label']] * senti['score']

# test on examples from huggingface website
sent_score("sent you a dm")

-0.5067996382713318

In [81]:
import os, json

json_file_path = "./sample_data/articles.json"

if os.path.exists(json_file_path):
    with open(json_file_path, "r") as json_file:
        data = json.load(json_file)
else:
    print(f"File '{json_file_path}' does not exist.")


In [22]:
import spacy
from spacy.lang.en import English
import re

# Load the spaCy model for English
nlp = spacy.load("en_core_web_sm")

In [76]:
def has_prop_noun(sentence):
  doc = nlp(sentence)
  for k in range(len(doc) - 1):
      token = doc[k]
      if token.pos_ == 'PROPN' and doc[k+1].pos_ == 'PROPN':  # PROPN is the tag for proper nouns in spaCy
          return True, token.text + " " + doc[k+1].text
  return False, ""

In [77]:
def get_athlete_scores(text, chunk_start=0 , max_sentences=1):
    # Split the text into sentences
    nlp = English()
    nlp.add_pipe('sentencizer')
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    athlete_scores = {}

    chunks = []
    current_chunk = []
    sentences_in_chunk = 0
    proper_noun_found = False

    chunks.append(" ".join(sentences[0:chunk_start]))

    for i in range(chunk_start % max_sentences, len(sentences)):
        sentence = sentences[i]
        has_proper_noun, name = has_prop_noun(sentence)

        # Rules for starting a new chunk
        if sentences_in_chunk >= max_sentences or (proper_noun_found and has_proper_noun):
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            sentences_in_chunk = 1
            proper_noun_found = has_proper_noun

            score = sent_score(current_chunk)
            if name in athlete_scores:
              athlete_scores[name] += score
            else:
              athlete_scores[name] = score
        else:
            current_chunk.append(sentence)
            sentences_in_chunk += 1
            if has_proper_noun:
                proper_noun_found = True

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return athlete_scores

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [85]:
# Load the dataset
BSS_data = pd.read_csv('sample_data/batting_season_summary.csv')
# free_agents = pd.read_csv('sample_data/submission_example.csv')

In [92]:
# Pre-initialize set from players list
# This ensures that extraneous proper nouns (ie, Major League) aren't added
  # to the set down the line
all_players_set = {}
for i, row in BSS_data.iterrows():
  all_players_set[row['Name']] = 0

for i in range(len(data)):
  scores = get_athlete_scores(data[i])
  # print(scores)
  for name in scores:
    if name in all_players_set:
      all_players_set[name] += scores[name]
      print("updated", name, "with", scores[name], "current", all_players_set[name])



updated Tony Torcato with 1.9978371262550354
updated Jim Abbott with 0.9991753697395325
updated Josh Hall with 0.9995265007019043
updated Anthony Shumaker with 1.9984532594680786
updated Marty McLeary with 0.9995501637458801
updated Andy Thompson with 0.9990772008895874
updated Brooks Raley with 0.9924343824386597
updated Brandon Dickson with 0.9569447040557861
updated P.J. Walters with 0.9336886405944824
updated Kevin Mattison with 1.9994592070579529
updated Chris Valaika with 0.999038815498352
updated Michael Crotta with 0.7483897805213928
updated Lance Davis with 0.9997498393058777
updated Tim Cooney with 0.9991497993469238
updated Brandon Harper with 1.9971638917922974
updated Kurt Miller with 0.9994074106216431
updated Ron Wright with 0.9978639483451843
updated Matt Boyd with 0.9963514804840088
updated Josh Smith with 0.9993085861206055
updated Scott Alexander with 0.9935566782951355
updated Justin Brunette with 0.9986306428909302
updated Kevin Ritz with 2.994318425655365
updated 

In [96]:
# all_players_set
cleaned_players_set = {key: all_players_set[key] for key in sorted(all_players_set)}
cleaned_players_set

{'Aaron Looper': 2.999210774898529,
 'Aaron Scheffer': 0,
 'Abel De Los Santos': 0,
 'Adalberto Mendez': 0,
 'Adam Duvall': 0,
 'Adam Greenberg': 0,
 'Adam Peterson': 0.9992018342018127,
 'Adam Shabala': 0,
 'Adam Wilk': 0,
 'Adrian Houser': 0,
 'Agustin Montero': 0,
 'Akeel Morris': 0,
 'Alan Johnson': 0,
 'Alberto Cabrera': 0,
 'Alex Castellanos': 0,
 'Alex Hernandez': 0,
 'Alex Meyer': 0,
 'Alex Pelaez': 0,
 'Allan Dykstra': 0,
 'Alvin Colina': 0,
 'Anderson Garcia': 0,
 'Andrew Albers': 0,
 'Andrew Brackman': 0,
 'Andrew Werner': 0,
 'Andy Barkett': 0,
 'Andy Cavazos': 0,
 'Andy Thompson': 0.9990772008895874,
 'Andy Van Hekken': 0,
 'Andy Wilkins': 0,
 'Angel Castro': 0.9998213648796082,
 'Angel Chavez': 0,
 'Angel Salome': 0.9998524188995361,
 'Anthony Claggett': 0.9997652173042297,
 'Anthony Ferrari': 0,
 'Anthony Shumaker': 1.9984532594680786,
 'Argenis Diaz': 0,
 'Ariel Pena': 0,
 'Arturo Lopez': 0,
 'Asher Wojciechowski': 0,
 'Atahualpa Severino': 0,
 'Audry Perez': 0.99972301

In [97]:
with open('output.txt', 'w') as file:
    for name in all_players_set:
        file.write(f'{name},{all_players_set[name]}\n')
